<a href="https://colab.research.google.com/github/MirianCabral/PySpark/blob/main/PySpark_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
!pip install pyspark

In [93]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [94]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_03')
    .getOrCreate()
)

In [95]:
df = spark.read.csv('/content/wc2018-players.csv', header= True, inferSchema=True)

In [96]:
df.show(5)

+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows



In [97]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao').withColumnRenamed('FIFA Popular Name', 'Nome_FIFA')\
.withColumnRenamed('Birth Date', 'Nascimento').withColumnRenamed('Shirt Name', 'Nome_Camiseta').withColumnRenamed('Club', 'Time')\
.withColumnRenamed('Height', 'Altura').withColumnRenamed('Weight', 'Peso')

dia = udf(lambda data: data.split('.')[0])
mes = udf(lambda data: data.split('.')[1])
ano = udf(lambda data: data.split('.')[2])

df = df.withColumn('Dia', dia('Nascimento')).withColumn('Mes', mes('Nascimento')).withColumn('Ano', ano('Nascimento'))
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia').cast(DateType()))

df.show(5)

+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+---+----+---------------+
|  Selecao|Numero|Posicao|         Nome_FIFA|Nascimento|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+------------------+----------+-------------+--------------------+------+----+---+---+----+---------------+
|Argentina|     3|     DF|TAGLIAFICO Nicolas|31.08.1992|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|    PAVON Cristian|21.01.1996|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|    LANZINI Manuel|15.02.1993|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|    SALVIO Eduardo|13.07.1990|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|
|Argentina|    10|     FW|      MESSI Lionel|24.06.1987|        MESSI

###Alteração Aula PySpark 02

In [98]:
df2 = df
df = df.drop('Nascimento', 'Nome_FIFA')

###Distinct()

In [99]:
df.select(col('Selecao')).distinct().show()

+--------------+
|       Selecao|
+--------------+
|        Russia|
|       Senegal|
|        Sweden|
|       IR Iran|
|       Germany|
|        France|
|     Argentina|
|       Belgium|
|          Peru|
|       Croatia|
|       Nigeria|
|Korea Republic|
|         Spain|
|       Denmark|
|       Morocco|
|        Panama|
|       Iceland|
|       Uruguay|
|        Mexico|
|       Tunisia|
+--------------+
only showing top 20 rows



###Collect()

In [100]:
lista = df.select(col('Selecao')).distinct().collect()
lista

[Row(Selecao='Russia'),
 Row(Selecao='Senegal'),
 Row(Selecao='Sweden'),
 Row(Selecao='IR Iran'),
 Row(Selecao='Germany'),
 Row(Selecao='France'),
 Row(Selecao='Argentina'),
 Row(Selecao='Belgium'),
 Row(Selecao='Peru'),
 Row(Selecao='Croatia'),
 Row(Selecao='Nigeria'),
 Row(Selecao='Korea Republic'),
 Row(Selecao='Spain'),
 Row(Selecao='Denmark'),
 Row(Selecao='Morocco'),
 Row(Selecao='Panama'),
 Row(Selecao='Iceland'),
 Row(Selecao='Uruguay'),
 Row(Selecao='Mexico'),
 Row(Selecao='Tunisia'),
 Row(Selecao='Saudi Arabia'),
 Row(Selecao='Switzerland'),
 Row(Selecao='Brazil'),
 Row(Selecao='Japan'),
 Row(Selecao='England'),
 Row(Selecao='Poland'),
 Row(Selecao='Portugal'),
 Row(Selecao='Australia'),
 Row(Selecao='Costa Rica'),
 Row(Selecao='Egypt'),
 Row(Selecao='Serbia'),
 Row(Selecao='Colombia')]

In [101]:
type(lista[20][0])

str

In [102]:
lista[1][0]

'Senegal'

In [103]:
paises = []
for pais in lista:
  paises.append(pais[0])

paises

['Russia',
 'Senegal',
 'Sweden',
 'IR Iran',
 'Germany',
 'France',
 'Argentina',
 'Belgium',
 'Peru',
 'Croatia',
 'Nigeria',
 'Korea Republic',
 'Spain',
 'Denmark',
 'Morocco',
 'Panama',
 'Iceland',
 'Uruguay',
 'Mexico',
 'Tunisia',
 'Saudi Arabia',
 'Switzerland',
 'Brazil',
 'Japan',
 'England',
 'Poland',
 'Portugal',
 'Australia',
 'Costa Rica',
 'Egypt',
 'Serbia',
 'Colombia']

###When / Otherwise()

In [104]:
df.withColumn('Coluna_Nova', when(col('Selecao')== "Argentina", 'Argentinos').otherwise('NÂO ARGENTINO')).show(50)

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+-------------+
|  Selecao|Numero|Posicao|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|  Coluna_Nova|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+-------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|   Argentinos|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|   Argentinos|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|   Argentinos|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|   Argentinos|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|   Argentinos|
|Argentina|     4|     DF|      ANSALDI|     Torino FC (ITA)|   

In [105]:
europa        = ['Sweden', 'Germany', 'France', 'Belgium', 'Croatia', 'Spain', 'Denmark', 'Iceland', 'Switzerland', 'England', 'Poland', 'Portugal', 'Serbia']
asia          = ['Russia', 'IR Iran', 'Korea Republic', 'Saudi Arabia', 'Japan']
africa        = ['Senegal', 'Nigeria', 'Morocco', 'Tunisia', 'Egypt']
oceania       = ['Australia']
america_norte = ['Panama', 'Mexico', 'Costa Rica']
america_sul   = ['Argentina', 'Peru', 'Uruguay', 'Brazil', 'Colombia']

In [106]:
df = df.withColumn('Continente', when(col('Selecao').isin(europa), 'Europa')\
              .when(col('Selecao').isin(asia), 'Asia')\
              .when(col('Selecao').isin(africa), 'Africa')\
              .when(col('Selecao').isin(oceania), 'Oceania')\
              .when(col('Selecao').isin(america_norte),'America do Norte')\
              .when(col('Selecao').isin(america_sul), 'America do Sul')\
              .otherwise('Verificar')
              )

In [107]:
df.show(50)

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|  Selecao|Numero|Posicao|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|    Continente|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|America do Sul|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|America do Sul|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|America do Sul|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|America do Sul|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|America do Sul|
|Argentina|     4|     DF|      ANSALDI|     Torino FC (

###Union (Concat)

In [108]:
df_america_sul = df.filter('Continente == "America do Sul"')
df_america_sul.select('Selecao'). distinct().show()

+---------+
|  Selecao|
+---------+
|Argentina|
|     Peru|
|  Uruguay|
|   Brazil|
| Colombia|
+---------+



In [109]:
df_america_norte = df.filter('Continente == "America do Norte"')
df_america_norte.select('Selecao'). distinct().show()

+----------+
|   Selecao|
+----------+
|    Panama|
|    Mexico|
|Costa Rica|
+----------+



In [110]:
df_americas = df_america_sul.union(df_america_norte)

In [111]:
df_americas.select('Selecao'). distinct().show()

+----------+
|   Selecao|
+----------+
| Argentina|
|      Peru|
|   Uruguay|
|    Brazil|
|  Colombia|
|    Panama|
|    Mexico|
|Costa Rica|
+----------+



In [112]:
df_america_sul = df_america_sul.drop('Continente')

In [113]:
df_america_sul.show()

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+
|  Selecao|Numero|Posicao|Nome_Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|
|Argentina|     4|     DF|      ANSALDI|     Torino FC (ITA)|   181|  73| 20| 09|1986|     1986-09-20|
|Argentina|     5|     MF|       BIGLIA|      AC Milan (ITA)|   175|  73|

###JOIN

In [114]:
arg = df.filter('Selecao == "Argentina"')
bra = df.filter('Selecao == "Brazil"')

In [115]:
arg = arg.drop ('Time', 'Dia', 'Mes', 'Ano', 'Continente', 'Peso', 'Data_Nascimento')
bra = bra.drop ('Time', 'Dia', 'Mes', 'Ano', 'Continente', 'Peso', 'Data_Nascimento')

###Join Simples

In [126]:
# Dados dos joins 100% correspondentes

In [116]:
dfnovo = arg.join(bra, arg.Numero == bra.Numero)
dfnovo.show(25)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|     3|     DF|   TAGLIAFICO|   169| Brazil|     3|     DF|      MIRANDA|   186|
|Argentina|    22|     MF|        PAVÓN|   169| Brazil|    22|     DF|       FAGNER|   168|
|Argentina|    15|     MF|      LANZINI|   167| Brazil|    15|     MF|     PAULINHO|   181|
|Argentina|    18|     DF|       SALVIO|   167| Brazil|    18|     MF|         FRED|   169|
|Argentina|    10|     FW|        MESSI|   170| Brazil|    10|     FW|    NEYMAR JR|   175|
|Argentina|     4|     DF|      ANSALDI|   181| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|     5|     MF|       BIGLIA|   175| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     7|     MF|       BANEGA|   175| Brazil|     7|     FW|     D. CO

In [117]:
arg = arg.withColumn('Numero', col('Numero') + 1)

###Inner Join

In [ ]:
# Mostra apenas os dados que tem correspondência

In [118]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'inner')
dfnovo.show(50)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|     4|     DF|   TAGLIAFICO|   169| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|    23|     MF|        PAVÓN|   169| Brazil|    23|     GK|      EDERSON|   188|
|Argentina|    16|     MF|      LANZINI|   167| Brazil|    16|     GK|       CASSIO|   195|
|Argentina|    19|     DF|       SALVIO|   167| Brazil|    19|     MF|      WILLIAN|   175|
|Argentina|    11|     FW|        MESSI|   170| Brazil|    11|     MF|  P. COUTINHO|   172|
|Argentina|     5|     DF|      ANSALDI|   181| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE LUIS|   182|
|Argentina|     8|     MF|       BANEGA|   175| Brazil|     8|     MF|   R. AUGU

In [119]:
dfnovo.count()

22

###Left Join

In [ ]:
# Mostra todos os lados dos DF do lado ESQUERDO
# (Valores do lado dir que não tiverem correspondência serão mostrados como nulos)

In [120]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'left')
dfnovo.show(50)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|     4|     DF|   TAGLIAFICO|   169| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|    23|     MF|        PAVÓN|   169| Brazil|    23|     GK|      EDERSON|   188|
|Argentina|    16|     MF|      LANZINI|   167| Brazil|    16|     GK|       CASSIO|   195|
|Argentina|    19|     DF|       SALVIO|   167| Brazil|    19|     MF|      WILLIAN|   175|
|Argentina|    11|     FW|        MESSI|   170| Brazil|    11|     MF|  P. COUTINHO|   172|
|Argentina|     5|     DF|      ANSALDI|   181| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE LUIS|   182|
|Argentina|     8|     MF|       BANEGA|   175| Brazil|     8|     MF|   R. AUGU

###Rigth Join

In [ ]:
# Mostra todos os lados dos DF do lado DIREITO
# (Valores do lado esq que não tiverem correspondência serão mostrados como nulos)

In [121]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'right')
dfnovo.show(50)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|    18|     DF|     OTAMENDI|   181| Brazil|    18|     MF|         FRED|   169|
|Argentina|    21|     MF|     LO CELSO|   177| Brazil|    21|     FW|       TAISON|   172|
|Argentina|    17|     DF|         ROJO|   189| Brazil|    17|     MF|  FERNANDINHO|   179|
|Argentina|    22|     FW|       DYBALA|   177| Brazil|    22|     DF|       FAGNER|   168|
|Argentina|    10|     FW|      HIGUAÍN|   184| Brazil|    10|     FW|    NEYMAR JR|   175|
|Argentina|    11|     FW|        MESSI|   170| Brazil|    11|     MF|  P. COUTINHO|   172|
|Argentina|     7|     DF|        FAZIO|   199| Brazil|     7|     FW|     D. COSTA|   182|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE L

###Full Join

In [ ]:
# Será mostrado todas as linhas
# (valores que não corresponderem de ambos os lados serão representados como NULOS)

In [122]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'full')
dfnovo.show(50)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|     NULL|  NULL|   NULL|         NULL|  NULL| Brazil|     1|     GK|    A. BECKER|   193|
|Argentina|     2|     GK|       GUZMÁN|   192| Brazil|     2|     DF|     T. SILVA|   183|
|Argentina|     3|     DF|      MERCADO|   181| Brazil|     3|     DF|      MIRANDA|   186|
|Argentina|     4|     DF|   TAGLIAFICO|   169| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|     5|     DF|      ANSALDI|   181| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE LUIS|   182|
|Argentina|     7|     DF|        FAZIO|   199| Brazil|     7|     FW|     D. COSTA|   182|
|Argentina|     8|     MF|       BANEGA|   175| Brazil|     8|     MF|   R. AUGU

###Semi Join

In [ ]:
# Similar ao Inner Join, porém apenas os dados do DataFrame esquerdo é mostrado

In [123]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'semi')
dfnovo.show(50)

+---------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+
|Argentina|     4|     DF|   TAGLIAFICO|   169|
|Argentina|    23|     MF|        PAVÓN|   169|
|Argentina|    16|     MF|      LANZINI|   167|
|Argentina|    19|     DF|       SALVIO|   167|
|Argentina|    11|     FW|        MESSI|   170|
|Argentina|     5|     DF|      ANSALDI|   181|
|Argentina|     6|     MF|       BIGLIA|   175|
|Argentina|     8|     MF|       BANEGA|   175|
|Argentina|    15|     DF|   MASCHERANO|   174|
|Argentina|    22|     FW|       DYBALA|   177|
|Argentina|    20|     FW|       AGÜERO|   172|
|Argentina|    10|     FW|      HIGUAÍN|   184|
|Argentina|    12|     MF|     DI MARÍA|   178|
|Argentina|    21|     MF|     LO CELSO|   177|
|Argentina|    14|     MF|         MEZA|   180|
|Argentina|     9|     DF|        ACUÑA|   172|
|Argentina|     3|     DF|      MERCADO|   181|
|Argentina|    18|     DF|     OTAMENDI|

###Anti Join

In [125]:
# Mostra os dados do DF do lado esquerdo que NÂO possuem correspondências

In [124]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'anti')
dfnovo.show(50)

+---------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome_Camiseta|Altura|
+---------+------+-------+-------------+------+
|Argentina|    24|     GK|    CABALLERO|   186|
+---------+------+-------+-------------+------+

